## Instalação Dependencias

In [12]:
pip install PyQt5

Note: you may need to restart the kernel to use updated packages.


## Classes

In [13]:
import math

class Window:
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        
    # def rotate(self, angle_degrees):
    #     angle = math.radians(angle_degrees)
        
    #     cx = (self.x_min + self.x_max) / 2
    #     cy = (self.y_min + self.y_max) / 2
        
    #     cos_theta = math.cos(angle)
    #     sin_theta = math.sin(angle)

    #     def rotate_point(x, y):
    #         x -= cx
    #         y -= cy
            
    #         x_new = x * cos_theta - y * sin_theta
    #         y_new = x * sin_theta + y * cos_theta
            
    #         x_new += cx
    #         y_new += cy
            
    #         return x_new, y_new

    #     self.x_min, self.y_min = rotate_point(self.x_min, self.y_min)
    #     self.x_max, self.y_max = rotate_point(self.x_max, self.y_max)
        
class Viewport:
    def __init__(self, x_min, y_min, x_max, y_max):
        self.x_min = x_min
        self.y_min = y_min
        self.x_max = x_max
        self.y_max = y_max
        
class Ponto:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def move(self, dx, dy):
        self.x += dx
        self.y += dy

    def resize(self, factor, center_x, center_y):
        self.x = center_x + (self.x - center_x) * factor
        self.y = center_y + (self.y - center_y) * factor

    def rotate(self, angle, center_x, center_y):
        
        rad = math.radians(angle)

        translated_x = self.x - center_x
        translated_y = self.y - center_y

        rotated_x = translated_x * math.cos(rad) - translated_y * math.sin(rad)
        rotated_y = translated_x * math.sin(rad) + translated_y * math.cos(rad)
        
        self.x = rotated_x + center_x
        self.y = rotated_y + center_y

    def __repr__(self):
        return f"Ponto({self.x}, {self.y})"

class Reta:
    def __init__(self, ponto1, ponto2):
        self.ponto1 = ponto1
        self.ponto2 = ponto2
    
    def move(self, dx, dy):
        self.ponto1.move(dx, dy)
        self.ponto2.move(dx, dy)

    def resize(self, factor, center_x, center_y):
        self.ponto1.x = center_x + (self.ponto1.x - center_x) * factor
        self.ponto1.y = center_y + (self.ponto1.y - center_y) * factor
        self.ponto2.x = center_x + (self.ponto2.x - center_x) * factor
        self.ponto2.y = center_y + (self.ponto2.y - center_y) * factor
    
    def rotate(self, angle, center_x, center_y):
        self.ponto1.rotate(angle, center_x, center_y)
        self.ponto2.rotate(angle, center_x, center_y)

    def __repr__(self):
        return f"Reta({self.ponto1}, {self.ponto2})"
        
class Poligono:
    def __init__(self, pontos):
        self.pontos = pontos 
    
    def move(self, dx, dy):
        for ponto in self.pontos:
            ponto.move(dx, dy)

    def resize(self, factor, center_x, center_y):
        for ponto in self.pontos:
            ponto.x = center_x + (ponto.x - center_x) * factor
            ponto.y = center_y + (ponto.y - center_y) * factor
            
    def rotate(self, angle, center_x, center_y):
        for ponto in self.pontos:
            ponto.rotate(angle, center_x, center_y)

    def __repr__(self):
        return f"Poligono({self.pontos})"

## Parser XML

In [14]:
import os
import xml.etree.ElementTree as ET

In [15]:
class Parser:
    @staticmethod
    def parse_xml_file(filename):
        tree = ET.parse(filename)
        root = tree.getroot()

        objetos = []
        
        vp_min = root.find("./viewport/vpmin")
        vp_max = root.find("./viewport/vpmax")
        viewport = Viewport(int(vp_min.attrib['x']), int(vp_min.attrib['y']), int(vp_max.attrib['x']), int(vp_max.attrib['y']))

    
        w_min = root.find("./window/wmin")
        w_max = root.find("./window/wmax")
        window = Window(float(w_min.attrib['x']), float(w_min.attrib['y']), float(w_max.attrib['x']), float(w_max.attrib['y']))

        for ponto in root.findall("./ponto"):
            objetos.append(Ponto(float(ponto.attrib['x']), float(ponto.attrib['y'])))

        for reta in root.findall("./reta"):
            ponto1 = reta.find("ponto[1]")
            ponto2 = reta.find("ponto[2]")
            objetos.append(Reta(Ponto(float(ponto1.attrib['x']), float(ponto1.attrib['y'])), Ponto(float(ponto2.attrib['x']), float(ponto2.attrib['y']))))

        for poligono in root.findall("./poligono"):
            pontos = []
            for ponto in poligono.findall("ponto"):
                pontos.append(Ponto(float(ponto.attrib['x']), float(ponto.attrib['y'])))
            objetos.append(Poligono(pontos))
            
        return viewport, window, objetos

    @staticmethod
    def write_to_xml_file(objects):
        root = ET.Element("objects")

        for obj in objects:
            if isinstance(obj, Ponto):
                point_elem = ET.SubElement(root, "point")
                point_elem.set("x", str(obj.x))
                point_elem.set("y", str(obj.y))
            elif isinstance(obj, Reta):
                line_elem = ET.SubElement(root, "line")
                point1_elem = ET.SubElement(line_elem, "point1")
                point1_elem.set("x", str(obj.ponto1.x))
                point1_elem.set("y", str(obj.ponto1.y))
                point2_elem = ET.SubElement(line_elem, "point2")
                point2_elem.set("x", str(obj.ponto2.x))
                point2_elem.set("y", str(obj.ponto2.y))
            elif isinstance(obj, Poligono):
                polygon_elem = ET.SubElement(root, "polygon")
                for point in obj.pontos:
                    point_elem = ET.SubElement(polygon_elem, "point")
                    point_elem.set("x", str(point.x))
                    point_elem.set("y", str(point.y))

        Parser.indent_xml_file(root)
        
        tree = ET.ElementTree(root)
        output_file = os.path.join("output", "saida.xml")
        tree.write(output_file, xml_declaration=True)
        
    @staticmethod
    def indent_xml_file(elem, level=0):
        indent_size = 4
        i = "\n" + level * " " * indent_size
        if len(elem):
            if not elem.text or not elem.text.strip():
                elem.text = i + " " * indent_size
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
            for elem in elem:
                Parser.indent_xml_file(elem, level + 1)
            if not elem.tail or not elem.tail.strip():
                elem.tail = i
        else:
            if level and (not elem.tail or not elem.tail.strip()):
                elem.tail = i


## Viewport transform objects

In [16]:
class ViewportTransform():
    def apply_transform(self, window, viewport, objects):
        transformed_objects = []
        
        for objeto in objects:
            if isinstance(objeto, Ponto):
                transformed_objects.append(self.transform_to_viewport_coordinates(viewport, window, objeto))
            elif isinstance(objeto, Reta):
                objeto.ponto1 = self.transform_to_viewport_coordinates(viewport, window, objeto.ponto1)
                objeto.ponto2 = self.transform_to_viewport_coordinates(viewport, window, objeto.ponto2)
                transformed_objects.append(objeto)
            elif isinstance(objeto, Poligono):
                pontos_transformados = [self.transform_to_viewport_coordinates(viewport, window, ponto) for ponto in objeto.pontos]
                objeto.pontos = pontos_transformados
                transformed_objects.append(objeto)

        return transformed_objects
    
    def transform_to_viewport_coordinates(self, viewport: Viewport, window: Window, ponto: Ponto):
            x_vp = ((ponto.x - window.x_min)/(window.x_max - window.x_min))*(viewport.x_max - viewport.x_min)
            y_vp = (1 - ((ponto.y - window.y_min)/(window.y_max - window.y_min)))*(viewport.y_max - viewport.y_min)
                
            return Ponto(x_vp, y_vp)
    

## Viewport

In [17]:
%matplotlib qt
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QLabel, QPushButton, QSpinBox, QDialog, QFormLayout, QLineEdit, QDialogButtonBox
from PyQt5.QtGui import QPainter, QPen, QPixmap
from PyQt5.QtCore import Qt

Dialogs

In [18]:
class PontoDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Adicionar Ponto")
        layout = QFormLayout()
        self.x_input = QLineEdit(self)
        self.y_input = QLineEdit(self)
        layout.addRow("X:", self.x_input)
        layout.addRow("Y:", self.y_input)
        self.buttons = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel, self)
        layout.addWidget(self.buttons)
        self.setLayout(layout)
        self.buttons.accepted.connect(self.accept)
        self.buttons.rejected.connect(self.reject)
    
    def get_data(self):
        return float(self.x_input.text()), float(self.y_input.text())

class RetaDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Adicionar Reta")
        layout = QFormLayout()
        self.x1_input = QLineEdit(self)
        self.y1_input = QLineEdit(self)
        self.x2_input = QLineEdit(self)
        self.y2_input = QLineEdit(self)
        layout.addRow("X1:", self.x1_input)
        layout.addRow("Y1:", self.y1_input)
        layout.addRow("X2:", self.x2_input)
        layout.addRow("Y2:", self.y2_input)
        self.buttons = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel, self)
        layout.addWidget(self.buttons)
        self.setLayout(layout)
        self.buttons.accepted.connect(self.accept)
        self.buttons.rejected.connect(self.reject)
    
    def get_data(self):
        return ((float(self.x1_input.text()), float(self.y1_input.text())),
               (float(self.x2_input.text()), float(self.y2_input.text())))

class PoligonoDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Adicionar Polígono")
        layout = QFormLayout()
        self.points_input = QLineEdit(self)
        layout.addRow("Pontos (x1,y1; x2,y2; ...):", self.points_input)
        self.buttons = QDialogButtonBox(QDialogButtonBox.Ok | QDialogButtonBox.Cancel, self)
        layout.addWidget(self.buttons)
        self.setLayout(layout)
        self.buttons.accepted.connect(self.accept)
        self.buttons.rejected.connect(self.reject)

    def get_data(self):
        points = self.points_input.text().split(";")
        cleaned_points = []
        for point in points:
            coords = point.strip("() ").split(",")
            if len(coords) == 2:
                try:
                    x, y = map(float, coords)
                    cleaned_points.append(Ponto(x, y))
                except ValueError:
                    continue
        return cleaned_points


In [19]:
import copy
import numpy as np

class Render(QMainWindow):
    def __init__(self, viewport: Viewport, windowInput: Window, objects):
        super().__init__()
        
        self.objects_transformation_matrix = np.identity(3)
        self.window_transformation_matrix = np.identity(3)

        self.setWindowTitle("Computação Gráfica")
        self.setGeometry(viewport.x_min, viewport.y_min, viewport.x_max + 30, viewport.y_max + 30)

        self.center_x = (windowInput.x_max - windowInput.x_min) / 2
        self.center_y = (windowInput.y_max - windowInput.y_min) / 2

        #window
        self.windowInput = windowInput
        
        #Coordernadas originais window
        self.objectsInput = objects

        #viewport
        self.label = QLabel(self)
        self.label.setGeometry(viewport.x_min, viewport.y_min, viewport.x_max, viewport.y_max)
        self.viewport = viewport
        
        self.pixmap = QPixmap(self.label.size())
        self.pixmap.fill(Qt.white)
        self.painter = QPainter(self.pixmap)
        pen = QPen(Qt.black)
        pen.setWidth(2)
        self.painter.setPen(pen)

        add_point_button = QPushButton('Add Ponto', self)
        add_point_button.move(500, 60)
        add_point_button.clicked.connect(self.open_point_dialog)
        
        add_line_button = QPushButton('Add Reta', self)
        add_line_button.move(500, 90)
        add_line_button.clicked.connect(self.open_line_dialog)
        
        add_polygon_button = QPushButton('Add Poligono', self)
        add_polygon_button.move(500, 120)
        add_polygon_button.clicked.connect(self.open_polygon_dialog)

        button_zoom_in = QPushButton('+', self)
        button_zoom_in.move(400, 0)
        button_zoom_in.clicked.connect(self.zoom_in)

        button_zoom_out = QPushButton('-', self)
        button_zoom_out.move(300, 0)
        button_zoom_out.clicked.connect(self.zoom_out)

        button_rotate_l = QPushButton('Rotate l', self)
        button_rotate_l.move(500, 0)
        button_rotate_l.clicked.connect(self.rotate_left)

        button_rotate_r = QPushButton('Rotate r', self)
        button_rotate_r.move(500, 30)
        button_rotate_r.clicked.connect(self.rotate_right)

        button_up = QPushButton('Up', self)
        button_up.move(100, 10)
        button_up.clicked.connect(self.move_up)

        button_left = QPushButton('Left', self)
        button_left.move(30, 52)
        button_left.clicked.connect(self.move_left)

        button_right = QPushButton('Rigth', self)
        button_right.move(170, 52)
        button_right.clicked.connect(self.move_right)

        button_down = QPushButton('Down', self)
        button_down.move(100, 95)
        button_down.clicked.connect(self.move_down)
    
        self.step_spinbox = QSpinBox(self)
        self.step_spinbox.setRange(1, 100)
        self.step_spinbox.setValue(1)
    
    def open_point_dialog(self):
        dialog = PontoDialog(self)
        if dialog.exec_():
            x, y = dialog.get_data()
            self.objectsInput.append(Ponto(x,y))
            self.update_display()

    def open_line_dialog(self):
        dialog = RetaDialog(self)
        if dialog.exec_():
            (x1, y1), (x2, y2) = dialog.get_data()
            self.objectsInput.append(Reta(Ponto(x1, y1),Ponto(x2, y2)))
            self.update_display()

    def open_polygon_dialog(self):
        dialog = PoligonoDialog(self)
        if dialog.exec_():
            pontos = dialog.get_data()
            # pontos = [Ponto(x, y) for x, y in dialog.get_data()]
            self.objectsInput.append(Poligono(pontos))
            self.update_display()
    
    def apply_transformation_to_window(self):
        windowCopy = copy.deepcopy(self.windowInput)

        corners = [
            np.array([windowCopy.x_min, windowCopy.y_min, 1]),
            np.array([windowCopy.x_max, windowCopy.y_min, 1]),
            np.array([windowCopy.x_max, windowCopy.y_max, 1]),
            np.array([windowCopy.x_min, windowCopy.y_max, 1])
        ]
        
        transformed_corners = [np.dot(self.window_transformation_matrix, corner) for corner in corners]
        
        x_min = min(c[0] for c in transformed_corners)
        y_min = min(c[1] for c in transformed_corners)
        x_max = max(c[0] for c in transformed_corners)
        y_max = max(c[1] for c in transformed_corners)
        
        return Window(x_min, y_min, x_max, y_max)
    
    def apply_transformation_to_objects(self, objects):
        transformed_objects = []

        for objeto in objects:
            if isinstance(objeto, Ponto):
                ponto_homogeneo = np.array([objeto.x, objeto.y, 1])
                ponto_transformado = self.objects_transformation_matrix @ ponto_homogeneo
                transformed_objects.append(Ponto(ponto_transformado[0], ponto_transformado[1]))

            elif isinstance(objeto, Reta):
                ponto1_homogeneo = np.array([objeto.ponto1.x, objeto.ponto1.y, 1])
                ponto2_homogeneo = np.array([objeto.ponto2.x, objeto.ponto2.y, 1])

                ponto1_transformado = self.objects_transformation_matrix @ ponto1_homogeneo
                ponto2_transformado = self.objects_transformation_matrix @ ponto2_homogeneo

                objeto.ponto1 = Ponto(ponto1_transformado[0], ponto1_transformado[1])
                objeto.ponto2 = Ponto(ponto2_transformado[0], ponto2_transformado[1])
                transformed_objects.append(objeto)

            elif isinstance(objeto, Poligono):
                pontos_transformados = []
                for ponto in objeto.pontos:
                    ponto_homogeneo = np.array([ponto.x, ponto.y, 1])
                    ponto_transformado = self.objects_transformation_matrix @ ponto_homogeneo
                    pontos_transformados.append(Ponto(ponto_transformado[0], ponto_transformado[1]))
                objeto.pontos = pontos_transformados
                transformed_objects.append(objeto)

        return transformed_objects

    def update_display(self):
        transformed_window = self.apply_transformation_to_window()
        viewport_transform = ViewportTransform()

        objects_copy = copy.deepcopy(self.objectsInput)
        objects_copy = self.apply_transformation_to_objects(objects_copy)

        objects_transformed = viewport_transform.apply_transform(transformed_window, self.viewport, objects_copy)
        self.cleanWindow()
        self.render_objects(objects=objects_transformed)
    
    def move_up(self):
        step = self.step_spinbox.value()
        self.move_window(0, step)
    
    def move_down(self):
        step = self.step_spinbox.value()
        self.move_window(0, -step)
    
    def move_left(self):
        step = self.step_spinbox.value()
        self.move_window(-step, 0)
    
    def move_right(self):
        step = self.step_spinbox.value()
        self.move_window(step, 0)

    def apply_matrix_to_point(self, matrix, point):
        point_vector = np.array([point.x, point.y, 1])
        transformed_vector = np.dot(matrix, point_vector)
        return Ponto(transformed_vector[0], transformed_vector[1])

    def rotate_objects(self, angle):
        rad = np.deg2rad(angle)
        
        cx, cy = self.center_x, self.center_y
        
        translate_to_origin = np.array([
            [1, 0, -cx],
            [0, 1, -cy],
            [0, 0, 1]
        ])
        
        rotation_matrix = np.array([
            [np.cos(rad), -np.sin(rad), 0],
            [np.sin(rad), np.cos(rad), 0],
            [0, 0, 1]
        ])
        
        translate_back = np.array([
            [1, 0, cx],
            [0, 1, cy],
            [0, 0, 1]
        ])
        
        rotation_transform = np.dot(np.dot(translate_back, rotation_matrix), translate_to_origin)
        
        self.objects_transformation_matrix = np.dot(rotation_transform, self.objects_transformation_matrix)
        
        self.update_display()

    def rotate_left(self):
        self.rotate_objects(10)

    def rotate_right(self):
        self.rotate_objects(-10)

    def resize_window(self, scale_factor):

        translate_to_origin = np.array([
            [1, 0, -self.center_x],
            [0, 1, -self.center_y],
            [0, 0, 1]
        ])

        scale_matrix = np.array([
            [scale_factor, 0, 0],
            [0, scale_factor, 0],
            [0, 0, 1]
        ])

        translate_back = np.array([
            [1, 0, self.center_x],
            [0, 1, self.center_y],
            [0, 0, 1]
        ])

        scale_transform = np.dot(np.dot(translate_back, scale_matrix), translate_to_origin)

        self.window_transformation_matrix = np.dot(scale_transform, self.window_transformation_matrix)

        self.update_display()

    def zoom_in(self):
        self.resize_window(scale_factor=0.9)
    
    def zoom_out(self):
        self.resize_window(scale_factor=1.1)

    def move_window(self, dx, dy):

        translate_matrix = np.array([
            [1, 0, dx],
            [0, 1, dy],
            [0, 0, 1]
        ])

        self.window_transformation_matrix = np.dot(translate_matrix, self.window_transformation_matrix)

        self.update_display()

    def draw_line(self, reta:Reta):
        pixmap = self.label.pixmap()
        if not pixmap:
            pixmap = QPixmap(self.label.size())
            pixmap.fill(Qt.white)

        painter = QPainter(pixmap)
        pen = QPen(Qt.black)
        pen.setWidth(2)
        painter.setPen(pen)
        
        painter.drawLine(int(reta.ponto1.x), int(reta.ponto1.y), int(reta.ponto2.x), int(reta.ponto2.y))
        self.label.setPixmap(pixmap)

        painter.end()

    def cleanWindow(self):
        pixmap = self.label.pixmap()
        if not pixmap:
            pixmap = QPixmap(self.label.size())
            pixmap.fill(Qt.white)

        pixmap.fill(Qt.white)
        
    def draw_polygon(self, poligono:Poligono):
            pontoOrigin = poligono.pontos[0]
            
            for i in range(0, int(len(poligono.pontos))):
                if i == len(poligono.pontos)-1:
                    self.draw_line(Reta(poligono.pontos[i], pontoOrigin))
                else:
                    self.draw_line(Reta(poligono.pontos[i], poligono.pontos[i+1]))
                
    def draw_point(self, ponto:Ponto):
        pixmap = self.label.pixmap()
        if not pixmap:
            pixmap = QPixmap(self.label.size())
            pixmap.fill(Qt.white)

        painter = QPainter(pixmap)
        pen = QPen(Qt.black)
        pen.setWidth(2)
        painter.setPen(pen)
        
        painter.drawPoint(int(ponto.x), int(ponto.y))
        self.label.setPixmap(pixmap)
        
        painter.end()
    
    def render_objects(self, objects):

        self.objects = objects
        for object in objects:
            if isinstance(object, Ponto):
                self.draw_point(object)
            elif isinstance(object, Reta):
                self.draw_line(object)
            elif isinstance(object, Poligono):
                self.draw_polygon(object)
        self.draw_viewport_border()

    def draw_viewport_border(self):

        pixmap = self.label.pixmap()
        if not pixmap:
            pixmap = QPixmap(self.label.size())
            pixmap.fill(Qt.white)
        
        painter = QPainter(pixmap)
        pen = QPen(Qt.red)
        pen.setWidth(3)
        painter.setPen(pen)
        
        # Desenha a borda ao redor da área do QLabel
        painter.drawRect(self.label.rect())
        
        self.label.setPixmap(pixmap)
        painter.end()

## Main

In [20]:
import copy

class main():

    viewport, windowInput, objetos = Parser.parse_xml_file("./input/entrada.xml")

    objetos_copy = copy.deepcopy(objetos)
    transform = ViewportTransform()
    transformed_objects = transform.apply_transform(viewport=viewport, objects=objetos_copy, window=windowInput)
    window = Render(viewport, windowInput, objects= objetos)
    window.render_objects(transformed_objects)
    window.show()
    Parser.write_to_xml_file(transformed_objects)

if __name__ == "main":
    main()